In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [200]:
import keras
from keras.models import load_model
from keras.losses import binary_crossentropy, mean_squared_error, mean_squared_error

In [53]:
import building, training, models, callbacks

In [54]:
from importlib import reload

In [205]:
reload(building)
reload(training)
reload(models)
reload(callbacks)

<module 'callbacks' from '/home/sergeymiller1996/docs/alissa/callbacks.py'>

In [189]:
model = models.model((40, 300))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context1 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context2 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context3 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context4 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
model_159 

In [29]:
emb = building.make_emb('datasets/ru.csv')

In [30]:
train = pd.read_csv('datasets/sents/train_sents')
val = pd.read_csv('datasets/sents/val_sents')
train = train.fillna('')
val = val.fillna('')

In [31]:
train.head()

,0,1,2,3,4,5,6,7
0,275027354274464,"адем , в конце концов я не принимала участия в...",это точно .,"а ты имеешь , самое непосредственное отношение...",0,все были весьма впечатлены твоей презентацией .,good,0.947139
1,275027354274464,"адем , в конце концов я не принимала участия в...",это точно .,"а ты имеешь , самое непосредственное отношение...",1,"нет . я не знала , что всё это связано .",good,0.882188
2,275027354274464,"адем , в конце концов я не принимала участия в...",это точно .,"а ты имеешь , самое непосредственное отношение...",2,но стоило тебе прийти . и она дает тебе заколк...,bad,0.749163
3,275027354274464,"адем , в конце концов я не принимала участия в...",это точно .,"а ты имеешь , самое непосредственное отношение...",3,"учитывая , что у тебя не было особой заинтерес...",good,0.865382
4,275027354274464,"адем , в конце концов я не принимала участия в...",это точно .,"а ты имеешь , самое непосредственное отношение...",4,и я не должна была подталкивать тебя ко всем э...,good,0.621098


In [32]:
val.shape

(7876, 8)

In [206]:
def run_model(train, val, emb, epochs=20, batch_size=64, sample_len=10000, load_file=None, save_file=None):
    if load_file is None:
        model = models.model((40, 300), eps=1e-5)
    else:
        model = load_model(load_file)

        model.compile(
            loss={'class_out': binary_crossentropy, 
                  'auto3' : mean_squared_error, 
                  'auto4': mean_squared_error},
            loss_weights = {'class_out': 1, 
                  'auto3' : 0, 
                  'auto4': 0},
            optimizer='rmsprop',
        )
    
    filepath = "weights/weights.{epoch:02d}-{val_loss:.3f}.hdf5"
    
    y_train = (train['6'] == 'good').values
    y_val = (val['6'] == 'good').values
    w_train = train['7']
    
    sample_train = building.shuffle_by_groups(train, '0', random_state=0)[:sample_len]

    model.fit_generator(training.flow(train, emb, batch_size, y_train, w_train),
                        steps_per_epoch=train.shape[0] / batch_size, 
                        epochs=epochs, 
                        shuffle=True,
                        callbacks=[callbacks.TelegramCallback(), 
                                callbacks.RocCallback(sample_train, val, emb),
                                callbacks.NDCGCallback(sample_train, val, emb),
                                keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=3, min_lr=1e-5),
                                keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, 
                                    save_best_only=False, save_weights_only=False, mode='auto', period=1)
                                  ],
                        validation_data=training.flow(val, emb, batch_size, y_train),
                        validation_steps=val.shape[0] / batch_size,
                       )

    if save_file is not None:
        model.save(save_file)

In [207]:
run_model(train, val, emb, load_file='weights/weights.06-2.292.hdf5')

Epoch 1/20
1108/1107 [==============================] - 339s 306ms/step - loss: 0.5368 - class_out_loss: 0.5137 - auto3_loss: 0.9561 - auto4_loss: 0.8353 - val_loss: 0.9785 - val_class_out_loss: 0.9547 - val_auto3_loss: 0.9770 - val_auto4_loss: 0.8560
Epoch 2/20
1108/1107 [==============================] - 302s 273ms/step - loss: 0.5136 - class_out_loss: 0.4884 - auto3_loss: 0.9654 - auto4_loss: 0.8447 - val_loss: 0.9847 - val_class_out_loss: 0.9581 - val_auto3_loss: 0.9757 - val_auto4_loss: 0.8546
Epoch 3/20
1108/1107 [==============================] - 251s 227ms/step - loss: 4.7504 - class_out_loss: 4.7208 - auto3_loss: 0.9655 - auto4_loss: 0.8446 - val_loss: 8.5274 - val_class_out_loss: 8.5023 - val_auto3_loss: 0.9771 - val_auto4_loss: 0.8553
Epoch 4/20
1108/1107 [==============================] - 304s 274ms/step - loss: 6.3211 - class_out_loss: 6.2982 - auto3_loss: 0.9655 - auto4_loss: 0.8447 - val_loss: 7.7430 - val_class_out_loss: 7.7206 - val_auto3_loss: 0.9776 - val_auto4_loss: